In [1]:
#-*-coding:utf-8-*-

# 연습문제 8-3: 유튜브 댓글 수집하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import openpyxl


In [2]:
driverpath = os.path.join(os.path.dirname(os.getcwd()),'module','chromedriver.exe')

s_path =os.path.join(os.path.dirname(os.path.abspath(os.getcwd())),'test','temps')
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)


In [3]:
query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 여행브이로그
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:5
3.각 동영상에서 추출할 댓글은 몇건입니까?: 6


In [4]:
os.makedirs(s_path + '/' + s + query_txt)

ff_name = s_path + '/' + s + query_txt + '\\' + s + '-' + query_txt + '.txt'
fc_name = s_path + '/' + s + query_txt + '\\' + s + '-' + query_txt + '.csv'
fx_name = s_path + '/' + s + query_txt + '\\' + s + '-' + query_txt + '.xls'

In [5]:
s_time = time.time()

드라이버실행

In [6]:
driver = webdriver.Chrome(driverpath)
driver.get('https://www.youtube.com')

In [7]:
time.sleep(3)
element = driver.find_element_by_name('search_query')
element.send_keys(query_txt)
element.submit()
time.sleep(2)

In [8]:
def scroll_down(driver,num,sleepnum=5):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    for i in range(num) :
        driver.execute_script("window.scrollBy(0,5000);") # 한페이지 20개씩 출력값
        time.sleep(sleepnum)

In [9]:
if reple_cnt <20 :
    r_cnt = 1
else :
    r_cnt = math.ceil(reple_cnt/20)
    
if cnt < 20 :
    c_cnt = 1
else :
    c_cnt = math.ceil(cnt/20)     
    scroll_down(driver,c_cnt)

    



In [10]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [11]:
contents =[]
count=0

In [12]:
for i in soup.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
    contents.append('https://www.youtube.com/' +i['href'])
    count +=1
    if count == cnt :
        break

In [291]:
wb = openpyxl.Workbook()

In [292]:
wb.save(fx_name)

In [13]:
name = []
source =[]
like=[]
url=[]
    
for j in range(0,len(contents)):
    # 동영상 목록 순차 접속
    driver.get(contents[j])
    time.sleep(4)
    
    print('{} 번째 동영상 크롤링'.format(j+1))
    scroll_down(driver,r_cnt,7)
    
    
    html = driver.page_source
    soup1 = BeautifulSoup(html,'html.parser')
    time.sleep(4)
    
    # 영상 제목
    title = soup1.select('#container > h1 > yt-formatted-string')[0].get_text()

    # 조회수
    view = soup1.select('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer')[0].get_text()
    view1 = view.replace(",","")
    view2 = int(re.search('\d+',view1).group())

    # 총 댓글수
    comment = soup1.select('#count > yt-formatted-string')[0].get_text()
    
    comment1 = comment.replace(",","")
    comment2 = int(re.search('\d+',comment1).group())
    
    # 유투버 명
    # 구독자수
    # 게시날짜
    # 좋아요 수
    # 싫어요 수

    check = reple_cnt
    
    # 댓글 추출 갯수 조정
    if check > comment2 :
        check = comment2
    
    
    count = 0
    for k in range(check):
        try:
            name_s = soup1.select('#author-text')[k].get_text().strip()
            source_s = soup1.select('#content-text')[k].get_text()
            like_s = soup1.select('#vote-count-middle')[k].get_text().strip()

            with open(ff_name,'a',encoding='UTF-8') as f:
                f.write("------------------------------------------------------------------"+"\n")
                f.write("\n***** {} 번째 영상의 {} 번째  댓글 *****\n".format(j+1,count+1))
                f.write("1.유투브 URL주소 : " + contents[j] + "\n")
                f.write("2.댓글 작성자 : " + name_s + "\n")
                f.write("3.댓글 내용 : " + source_s + "\n")
                f.write("4.댓글 좋아요수 : " + like_s + "\n\n")
            count +=1
                


            # 작성자 name
            name.append(name_s)
            # 댓글 원문
            source.append(source_s)
            # 댓글 좋아요수
            like.append(like_s)
            # URL
            url.append(contents[j])
        
        except:
            pass

1 번째 동영상 크롤링
2 번째 동영상 크롤링
3 번째 동영상 크롤링
4 번째 동영상 크롤링
5 번째 동영상 크롤링


In [14]:
youtube_comments=pd.DataFrame()

In [15]:
youtube_comments['URL'] = pd.Series(url)
youtube_comments['댓글 작성자'] = pd.Series(name)
youtube_comments['댓글 내용'] = pd.Series(source)
youtube_comments['댓글 좋아요수'] = pd.Series(like)

In [20]:
# scv파일저장
youtube_comments.to_csv(fc_name,encoding="utf-8-sig",index=True)

In [19]:
# xlsx 저장
youtube_comments.to_excel(fx_name,
             sheet_name=query_txt,
             na_rep = '알수없음',
             index = False,
             header =True)

In [251]:
html = driver.page_source
soup1 = BeautifulSoup(html,'html.parser')
